In [1]:
import pandas as pd

!pip install thefuzz[levenshtein]

import thefuzz

from thefuzz import fuzz
from thefuzz import process

In [2]:
# loading in the excel files we just exported in the previous workbook
eia_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\filtered_grouped_eia.xlsx"
eia = pd.read_excel(eia_path)

ipm_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\filtered_grouped_ipm.xlsx"
ipm = pd.read_excel(ipm_path)

In [3]:
# Turning all letters in the data frame to lowercase so they can match easier 
eia = eia.applymap(lambda x: x.lower() if isinstance(x, str) else x)
ipm = ipm.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\63141\AppData\Local\Temp\ipykernel_19192\2246793782.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  eia = eia.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\63141\AppData\Local\Temp\ipykernel_19192\2246793782.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ipm = ipm.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [5]:
def fuzzy_merge(df1, df2, key1, key2, county_key, state_key, technology_key, threshold=85, limit=1):
    """
    Perform fuzzy matching and merge between two DataFrames based on plant names,
    but only if the county, state, and technology type match. Each plant name can only match once.

    df1, df2: DataFrames to merge
    key1, key2: Column names for plant names to merge on
    county_key: Column name for counties to match
    state_key: Column name for states to match
    technology_key: Column name for technology to match
    threshold: Minimum similarity score to consider a match
    limit: Number of matches to return (best match)
    """
    matched_names = set()
    used_names = set()  # To keep track of names already used in matches

    def match_row(row):
        # Filter df2 based on matching county, state, and technology type
        filtered_df2 = df2[
            (df2[county_key] == row[county_key]) &
            (df2[state_key] == row[state_key]) &
            (df2[technology_key] == row[technology_key])
        ]

        # Apply fuzzy matching on the filtered DataFrame
        if not filtered_df2.empty and pd.notnull(row[key1]):
            s = filtered_df2[key2].tolist()
            # Filter out names already used in previous matches
            s = [name for name in s if name not in used_names]
            if not s:
                return pd.Series([None, None], index=['best_match', 'match_score'])

            best_match = process.extractOne(row[key1], s, scorer=fuzz.token_set_ratio)
            if best_match and best_match[1] >= threshold:
                best_match_name = best_match[0]
                used_names.add(best_match_name)  # Mark this name as used
                return pd.Series([best_match_name, best_match[1]], index=['best_match', 'match_score'])

        return pd.Series([None, None], index=['best_match', 'match_score'])

    # Apply matching function
    match_results = df1.apply(match_row, axis=1)

    # Rename columns to avoid overlap
    match_results.columns = ['match_best_match', 'match_match_score']

    # Append match results to df1
    df1 = df1.join(match_results)

    # Perform the merge based on the best matches found
    merged_df = pd.merge(df1, df2, left_on='match_best_match', right_on=key2, how='left', suffixes=('', '_df2'))

    # Drop the 'match_best_match' column if it exists
    if 'match_best_match' in merged_df.columns:
        merged_df = merged_df.drop(columns=['match_best_match'])

    # Add capacity difference column if both capacity columns exist
    if 'Summer Capacity (MW)' in merged_df.columns and 'Summer Dispatchable' in merged_df.columns:
        merged_df['Capacity Difference (MW)'] = merged_df['Summer Capacity (MW)'] - merged_df['Summer Dispatchable']

    return merged_df

# Example usage with your DataFrames:
merged_ipm_eia = fuzzy_merge(eia, ipm, 
                              'Plant Name', 'Plant Name', 'County', 'State', 'Merged Capacity Types', 
                              threshold=85)

In [6]:
MS_90 = merged_ipm_eia[merged_ipm_eia['match_match_score'] >= 90]
MS_90.shape

(291, 17)

In [7]:
path = "C:\\Users\\63141\\Desktop\\github_files_v2\\GMR1.xlsx"
MS_90.to_excel(path, index = False)

In [9]:
# subtracting out the greater than 90 / no capacity mismatch plants from the orginal data frame we first brought in
unmatched_plants_ipm = ipm[~ipm['key'].isin(MS_90['key_df2'])]
unmatched_plants_eia = eia[~eia['key'].isin(MS_90['key'])]

In [11]:
path1 = "C:\\Users\\63141\\Desktop\\github_files_v2\\unmatched_ipm_R1.xlsx"
unmatched_plants_ipm.to_excel(path1, index = False)

path2 = "C:\\Users\\63141\\Desktop\\github_files_v2\\unmatched_eia_R1.xlsx"
unmatched_plants_eia.to_excel(path2, index = False)